In [2]:
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
import numpy as np
import json
import requests
import pandas as pd

In [4]:
df = pd.read_csv('../data/data.csv')

sample = df.sample(n=450)
sample.to_csv('../data/sample.csv', index=False)



In [6]:

sample[0:150].to_csv('../data/bia.csv', index=False)
sample[150:300].to_csv('../data/joni.csv', index=False)
sample[300:450].to_csv('../data/maggy.csv', index=False)


In [ ]:
CHRISTMAS_QRELS_FILE = '../qrels_files/christmas_movies.txt'
TWENTIES_QRELS_FILE = '../qrels_files/20s_movies.txt'
ROMANCE_TEEN_QRELS_FILE = '../qrels_files/romance_teen.txt'
SPACE_QRELS_FILE = '../qrels_files/space_movies.txt'

CHRISTMAS_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=primaryTitle%3A%20christmas%5E5.0%20synopsis%3Achristmas%5E3&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=christmas%5E2%20santa%5E2%20snow%20elf%20rodolf%20festive%20claus%20merry%20holliday%20%22candy%20cane%22%20%22christmas%20tree%22%20eve%20advent&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'
TWENTIES_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=primaryTitle%3A%20christmas%5E5.0%20synopsis%3Achristmas%5E3&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=christmas%5E2%20santa%5E2%20snow%20elf%20rodolf%20festive%20claus%20merry%20holliday%20%22candy%20cane%22%20%22christmas%20tree%22%20eve%20advent&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'
ROMANCE_TEEN_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=genres%3Adrama%20genres%3Aromance&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=romance%20teen%20crush%20heart-break%20%5C%22in%20love%5C%22%20high-school%20college%20friends%20friendship%20campus%20gossip%20passion%20attraction&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'
SPACE_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=genres%3Asci-fi%5E3.0%20synopsis%3Astars&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=space%20astronaut%20galaxy%20planets&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'

In [ ]:
# Read qrels to extract relevant documents
relevant = list(map(lambda el: el.strip(), open(CHRISTMAS_QRELS_FILE).readlines()))
# Get query results from Solr instance
results = requests.get(CHRISTMAS_QUERY_URL).json()['response']['docs']
